In [1]:
from utils.database_connection import DatabaseConnection
import utils.results_repository as rep
import pandas as pd
import numpy as np
from utils.column_name_utils import *
import utils.query_utils as query

In [2]:
db = DatabaseConnection()

Connected to MySQL Server version  8.0.31-23


In [3]:
experiment_name = "same-cpp-multi-core-micro-benchmark-different-compiler"
experiment_number = 1

used_dut_name = [
    'workstationone'
]

used_dut_env = 'prod'

used_test_case_names = [
    'fannkuch-redux.exe',
    'mandelbrot.exe'
]

used_compilers = [
    'clang'
]

used_thread = [
    'multi-threaded'
]

used_test_case_name = 'idle-case.exe'

ipg = "ipg"
lhm = "lhm"

used_mi_names = [
    ipg, lhm
]

ipg_sample_rate = 333
lhm_sample_rate = 100

used_min_temp = 0
used_max_temp = 200
used_burn_in = 1
used_allocated_cores = [
    []
]

### DUTs

In [4]:
used_dut_name_str = "','".join(used_dut_name)

In [5]:
dut = rep.get_devices_under_test(db)

dut

,Id,Name,Os,Env
0,1,workstationone,win32nt,prod


In [6]:
used_dut = dut.query(f"{ENV} == '{used_dut_env}' and {NAME} in ('{used_dut_name_str}')")

### Test Cases

In [7]:
tc = rep.get_test_cases(db)

tc

,Id,Name,Compiler,Language,Optimizations,TestCaseSize,Parameter,Threads
0,1,fannkuch-redux.exe,clang,c++,-openmp,microbenchmarks,12,multi-threaded
1,2,mandelbrot.exe,clang,c++,-openmp,microbenchmarks,16000,multi-threaded
2,3,idle-case.exe,clang,c++,-openmp,microbenchmarks,,multi-threaded


In [8]:
used_test_case_name_str = "','".join(used_test_case_names)
used_compiler_str = "','".join(used_compilers)
used_thread_str = "','".join(used_thread)

used_tc = tc.query(f"{COMPILER} in ('{used_compiler_str}') and {NAME} in ('{used_test_case_name_str}') and {THREADS} in ('{used_thread_str}')")

In [9]:
used_idle_case = tc.query(f"{COMPILER} in ('{used_compiler_str}') and {NAME} == '{used_test_case_name}' and {THREADS} in ('{used_thread_str}')")

### Measuring Instruments

In [10]:
mi = rep.get_measuring_instruments(db)

mi

,Id,Name,SampleRate
0,1,lhm,100
1,2,ipg,333


In [11]:
used_mi = mi.query(f"({NAME} == '{ipg}' and {SAMPLE_RATE} == {ipg_sample_rate}) or ({NAME} == '{lhm}' and {SAMPLE_RATE} == {lhm_sample_rate})")

### Configurations

In [12]:
used_allocated_cores_str = "','".join([str(x) for x in used_allocated_cores])

In [13]:
config = rep.get_configurations(db)

config

,Id,MinTemp,MaxTemp,Burnin,AllocatedCores
0,1,0,200,1,[]


In [14]:
used_configs = config.query(f"{MIN_TEMP} == {used_min_temp} and " +
                            f"{MAX_TEMP} == {used_max_temp} and " +
                            f"{BURN_IN} == {used_burn_in} and " +
                            f"{ALLOCATED_CORES} in ('{used_allocated_cores_str}')")

### Experiment Names

In [15]:
exp = rep.get_distinct_experiments(db)

exp

,Experiment,Name
0,1,same-cpp-multi-core-micro-benchmark-different-...


## Used Things

In [16]:
used_dut

,Id,Name,Os,Env
0,1,workstationone,win32nt,prod


In [17]:
used_tc

,Id,Name,Compiler,Language,Optimizations,TestCaseSize,Parameter,Threads
0,1,fannkuch-redux.exe,clang,c++,-openmp,microbenchmarks,12,multi-threaded
1,2,mandelbrot.exe,clang,c++,-openmp,microbenchmarks,16000,multi-threaded


In [18]:
used_mi

,Id,Name,SampleRate
0,1,lhm,100
1,2,ipg,333


In [19]:
used_idle_case

,Id,Name,Compiler,Language,Optimizations,TestCaseSize,Parameter,Threads
2,3,idle-case.exe,clang,c++,-openmp,microbenchmarks,,multi-threaded


In [20]:
used_configs

,Id,MinTemp,MaxTemp,Burnin,AllocatedCores
0,1,0,200,1,[]


In [26]:
for dut_index, dut_row in used_dut.iterrows():
    dut_id = dut_row[ID]
    for mi_index, mi_row in used_mi.iterrows():
        mi_id = mi_row[ID]
        for config_index, config_row in used_configs.iterrows():
            config_id = config_row[ID]
            for tc_index, tc_row in used_tc.iterrows():
                tc_id = tc_row[ID]
                tc_compiler = tc_row[COMPILER]
                tc_language = tc_row[LANGUAGE]
                tc_optimizations = tc_row[OPTIMIZATIONS]
                tc_test_case_size = tc_row[TEST_CASE_SIZE]
                tc_threads = tc_row[THREADS]
                
                tc_idle_case = query.get_idle_case(used_idle_case, tc_compiler, tc_language, tc_optimizations, tc_test_case_size, tc_threads)
                tc_idle_case_id =int(tc_idle_case[ID].iloc[0])
                
                tc_collection = rep.get_collections(db, dut_id, mi_id, config_id, tc_id, experiment_name, experiment_number)
                tc_idle_case_collection = rep.get_collections(db, dut_id, mi_id, config_id, tc_idle_case_id, experiment_name, experiment_number)
                
                tc_collection_id = int(tc_collection[ID].iloc[0])
                tc_idle_case_collection_id = int(tc_idle_case_collection[ID].iloc[0])
                
                tc_measurements = rep.get_measurements(db, tc_collection_id)
                tc_idle_case_measurements = rep.get_measurements(db, tc_idle_case_collection_id)

In [27]:
tc_measurements

,Id,Iteration,CollectionId,PackageTemperatureBegin,PackageTemperatureEnd,Duration,DramEnergyInJoules,GpuEnergyInJoules,CpuEnergyInJoules,BeginTime,EndTime,AdditionalMetadata
0,5,1,5,33.888889,34.222221,31213.0,18.253723,27.519165,1506.016968,2023-03-09 12:28:22,2023-03-09 12:28:53,{}
1,8,2,5,33.555557,34.555557,31198.0,18.331665,27.386841,1507.981567,2023-03-09 12:30:00,2023-03-09 12:30:31,{}
2,11,3,5,35.555557,38.333332,31189.0,18.389038,28.334961,1512.671631,2023-03-09 12:31:40,2023-03-09 12:32:11,{}
3,14,4,5,34.111111,34.888889,31290.0,18.299011,28.349426,1509.482544,2023-03-09 12:33:20,2023-03-09 12:33:51,{}
4,17,5,5,33.888889,34.666668,31176.0,18.242554,28.386658,1506.950928,2023-03-09 12:34:58,2023-03-09 12:35:29,{}
5,20,6,5,34.111111,34.444443,31160.0,18.260132,28.644897,1507.127686,2023-03-09 12:36:38,2023-03-09 12:37:09,{}
6,23,7,5,33.888889,34.888889,31215.0,18.258179,28.543945,1508.011597,2023-03-09 12:38:18,2023-03-09 12:38:49,{}
7,26,8,5,33.888889,34.888889,31163.0,18.234314,27.556213,1505.334839,2023-03-09 12:39:59,2023-03-09 12:40:30,{}
8,29,9,5,35.222221,36.444443,31247.0,18.390930,27.031311,1508.424072,2023-03-09 12:41:37,2023-03-09 12:42:08,{}
9,32,10,5,34.222221,36.000000,31157.0,18.230713,27.864685,1506.925293,2023-03-09 12:43:14,2023-03-09 12:43:45,{}


In [28]:
tc_idle_case_measurements

,Id,Iteration,CollectionId,PackageTemperatureBegin,PackageTemperatureEnd,Duration,DramEnergyInJoules,GpuEnergyInJoules,CpuEnergyInJoules,BeginTime,EndTime,AdditionalMetadata
0,6,1,6,34.333332,33.666668,15027.0,8.765503,12.247192,142.845337,2023-03-09 12:28:56,2023-03-09 12:29:11,{}
1,9,2,6,35.555557,33.888889,15036.0,9.148743,13.195923,146.283691,2023-03-09 12:30:34,2023-03-09 12:30:49,{}
2,12,3,6,35.666668,35.888889,15038.0,11.266541,12.791748,247.106506,2023-03-09 12:32:14,2023-03-09 12:32:29,{}
3,15,4,6,34.333332,34.222221,15034.0,8.772522,12.169250,144.436279,2023-03-09 12:33:54,2023-03-09 12:34:09,{}
4,18,5,6,34.333332,34.444443,15035.0,8.755859,12.980591,143.441772,2023-03-09 12:35:31,2023-03-09 12:35:46,{}
5,21,6,6,34.333332,34.000000,15027.0,8.756042,13.196899,143.459656,2023-03-09 12:37:11,2023-03-09 12:37:26,{}
6,24,7,6,34.333332,35.777779,15029.0,8.838562,12.200134,160.492249,2023-03-09 12:38:51,2023-03-09 12:39:06,{}
7,27,8,6,34.666668,34.222221,15042.0,8.758118,10.343018,140.798401,2023-03-09 12:40:33,2023-03-09 12:40:48,{}
8,30,9,6,35.777779,34.000000,15036.0,8.821655,10.841187,150.863403,2023-03-09 12:42:10,2023-03-09 12:42:25,{}
9,33,10,6,34.444443,34.111111,15027.0,8.760803,10.290405,140.634583,2023-03-09 12:43:47,2023-03-09 12:44:02,{}
